# Deep Learning

In [1]:
from keras.layers import Input, Dense, Lambda, Layer, LSTM, RepeatVector, Activation
from keras.models import Model
from keras.layers.core import Dropout
from keras import regularizers
import keras
import pandas as pd
import numpy as np
from keras import backend as K
from keras import metrics
from collections import namedtuple
pd.set_option("display.max_rows",35)
%matplotlib inline

Using TensorFlow backend.


In [2]:
kdd_train_2labels = pd.read_pickle("dataset/kdd_train_2labels.pkl")
kdd_test_2labels = pd.read_pickle("dataset/kdd_test_2labels.pkl")

#y_train_labels = pd.read_pickle("dataset/kdd_train_2labels_y.pkl")
#y_train_labels = pd.read_pickle("dataset/kdd_train_2labels.pkl")
#y_test_labels = pd.read_pickle("dataset/kdd_test_2labels_y.pkl")

output_columns_2labels = ['is_Attack','is_Normal']

from sklearn import model_selection as ms
from sklearn import preprocessing as pp

x_input = kdd_train_2labels.drop(output_columns_2labels, axis = 1)
y_output = kdd_train_2labels.loc[:,output_columns_2labels]

ss = pp.StandardScaler()
x_input = ss.fit_transform(x_input)

#le = pp.LabelEncoder()
#y_train = le.fit_transform(y_train_labels).reshape(-1, 1)
#y_test = le.transform(y_test_labels).reshape(-1, 1)

y_train = kdd_train_2labels.loc[:,output_columns_2labels].values

x_train, x_valid, y_train, y_valid = ms.train_test_split(x_input, 
                              y_train, 
                              test_size=0.1)
#x_valid, x_test, y_valid, y_test = ms.train_test_split(x_valid, y_valid, test_size = 0.4)

x_test = kdd_test_2labels.drop(output_columns_2labels, axis = 1)
y_test = kdd_test_2labels.loc[:,output_columns_2labels].values

x_test = ss.transform(x_test)

#x_train = np.hstack((x_train, y_train))
#x_valid = np.hstack((x_valid, y_valid))

#x_test = np.hstack((x_test, np.random.normal(loc = 0, scale = 0.01, size = y_test.shape)))

In [3]:
input_dim = 122
intermediate_dim = 10
latent_dim = 32
batch_size = 1409
hidden_layers = 8
classes = 2
drop_prob = 0.4
timesteps = 1

class Train:
    def build_lstm_model():
        Train.x = Input(shape=(timesteps, input_dim))
        encoded = LSTM(input_dim)(Train.x)
        encoded = Dropout(drop_prob)(encoded)
        decoded = RepeatVector(timesteps)(encoded)
        Train.y = LSTM(classes, return_sequences=True)(decoded)
        Train.y = Activation('softmax')(Train.y)

Train.build_lstm_model()

In [9]:
import itertools
#features_arr = [4, 16, 32, 256, 1024]
#hidden_layers_arr = [2, 6, 10, 100]

#features_arr = [4, 16, 32]
#hidden_layers_arr = [2, 6, 10]

features_arr = [1] # [4, 16, 32]
hidden_layers_arr = [6, 8, 16, 32, 64]

epoch_arr = [30]

score = namedtuple("score", ['epoch', 'no_of_features','hidden_layers','train_score', 'test_score'])
scores = []
predictions = {}

for e, h, f in itertools.product(epoch_arr, hidden_layers_arr, features_arr):
    
    print(" \n Current Layer Attributes - epochs:{} hidden layers:{} features count:{}".format(e,h,f))
    latent_dim = f
    epochs = e
    hidden_layers = h
    
    train_size = x_train.shape[0] - x_train.shape[0]%batch_size
    valid_size = x_valid.shape[0] - x_valid.shape[0]%batch_size

    
    #optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-04, decay=0.1)
    optimizer = keras.optimizers.RMSprop(lr=0.0001, decay=0.1)
    
    seq2seq_model = Model(Train.x, Train.y)
    seq2seq_model.compile(optimizer = optimizer, 
                      loss = keras.losses.categorical_crossentropy, 
                      metrics = ['accuracy'])
    
    ckp = keras.callbacks.ModelCheckpoint("dataset/epochs_{}_hidden layers_{}_features count_{}".format(e,h,f), 
                                          monitor='val_acc', verbose=1, 
                                          save_best_only=True, save_weights_only=False, 
                                          mode='auto', period=1)

    seq2seq_model.fit(x = x_train[:train_size,np.newaxis,:], y = y_train[:train_size,np.newaxis,:],
                 shuffle=True, epochs=epochs, 
                  batch_size = batch_size, 
                  validation_data = (x_test[:,np.newaxis,:], y_test[:,np.newaxis,:]),
                  verbose = 1, callbacks=[ckp])

    score_train = seq2seq_model.evaluate(x_valid[:valid_size,np.newaxis,:], y = y_valid[:valid_size,np.newaxis,:],
                               batch_size = batch_size,
                               verbose = 1)
    
    score_test = seq2seq_model.evaluate(x_test[:,np.newaxis,:], y = y_test[:,np.newaxis,:],
                           batch_size = batch_size,
                           verbose = 1)
    
    y_test_pred = seq2seq_model.predict(x_test[:,np.newaxis,:], batch_size=batch_size)
    y_test_pred = np.squeeze(y_test_pred)

    y_pred = y_test_pred #np.argmax(y_test_pred[:,-2:], axis = 1)
    
    curr_pred = pd.DataFrame({"Attack_prob":y_pred[:,0], "Normal_prob":y_pred[:,1]})
    predictions.update({"{}_{}_{}".format(e,f,h):curr_pred})
    
    scores.append(score(e,f,h,score_train[-1], score_test[-1])) #score_test[-1]))
    
    print("\n Train Acc: {}, Test Acc: {}".format(score_train[-1], 
                                                  score_test[-1])  )
    
scores = pd.DataFrame(scores)

 
 Current Layer Attributes - epochs:30 hidden layers:6 features count:1
Train on 112720 samples, validate on 22544 samples
Epoch 1/30
112720/112720 [==============================] - 5s - loss: 0.2248 - acc: 0.9799 - val_loss: 0.5729 - val_acc: 0.7421
Epoch 2/30
112720/112720 [==============================] - 2s - loss: 0.2247 - acc: 0.9802 - val_loss: 0.5729 - val_acc: 0.7421
Epoch 3/30
112720/112720 [==============================] - 2s - loss: 0.2248 - acc: 0.9800 - val_loss: 0.5729 - val_acc: 0.7421
Epoch 4/30
112720/112720 [==============================] - 2s - loss: 0.2248 - acc: 0.9801 - val_loss: 0.5728 - val_acc: 0.7421
Epoch 5/30
112720/112720 [==============================] - 2s - loss: 0.2246 - acc: 0.9802 - val_loss: 0.5728 - val_acc: 0.7421
Epoch 6/30
112720/112720 [==============================] - 2s - loss: 0.2248 - acc: 0.9800 - val_loss: 0.5728 - val_acc: 0.7421
Epoch 7/30
112720/112720 [==============================] - 3s - loss: 0.2249 - acc: 0.9799 - val_loss

112720/112720 [==============================] - 5s - loss: 0.2248 - acc: 0.9800 - val_loss: 0.5730 - val_acc: 0.7418
Epoch 2/30
112720/112720 [==============================] - 3s - loss: 0.2248 - acc: 0.9800 - val_loss: 0.5730 - val_acc: 0.7418
Epoch 3/30
112720/112720 [==============================] - 3s - loss: 0.2248 - acc: 0.9801 - val_loss: 0.5729 - val_acc: 0.7419
Epoch 4/30
112720/112720 [==============================] - 3s - loss: 0.2248 - acc: 0.9801 - val_loss: 0.5729 - val_acc: 0.7419
Epoch 5/30
112720/112720 [==============================] - 2s - loss: 0.2248 - acc: 0.9800 - val_loss: 0.5729 - val_acc: 0.7418
Epoch 6/30
112720/112720 [==============================] - 2s - loss: 0.2247 - acc: 0.9800 - val_loss: 0.5729 - val_acc: 0.7418
Epoch 7/30
112720/112720 [==============================] - 2s - loss: 0.2247 - acc: 0.9801 - val_loss: 0.5729 - val_acc: 0.7418
Epoch 8/30
112720/112720 [==============================] - 2s - loss: 0.2247 - acc: 0.9801 - val_loss: 0.57

112720/112720 [==============================] - 2s - loss: 0.2247 - acc: 0.9800 - val_loss: 0.5725 - val_acc: 0.7421
Epoch 4/30
112720/112720 [==============================] - 2s - loss: 0.2247 - acc: 0.9800 - val_loss: 0.5725 - val_acc: 0.7421
Epoch 5/30
112720/112720 [==============================] - 3s - loss: 0.2247 - acc: 0.9801 - val_loss: 0.5725 - val_acc: 0.7421
Epoch 6/30
112720/112720 [==============================] - 3s - loss: 0.2246 - acc: 0.9802 - val_loss: 0.5725 - val_acc: 0.7421
Epoch 7/30
112720/112720 [==============================] - 3s - loss: 0.2247 - acc: 0.9800 - val_loss: 0.5725 - val_acc: 0.7421
Epoch 8/30
112720/112720 [==============================] - 3s - loss: 0.2248 - acc: 0.9800 - val_loss: 0.5725 - val_acc: 0.7421
Epoch 9/30
112720/112720 [==============================] - 3s - loss: 0.2246 - acc: 0.9801 - val_loss: 0.5725 - val_acc: 0.7421
Epoch 10/30
112720/112720 [==============================] - 3s - loss: 0.2247 - acc: 0.9802 - val_loss: 0.5

In [10]:
scores.sort_values("test_score", ascending=False)

,epoch,no_of_features,hidden_layers,train_score,test_score
4,30,1,64,0.981991,0.742237
3,30,1,32,0.981991,0.742149
0,30,1,6,0.981991,0.742104
2,30,1,16,0.981991,0.742104
1,30,1,8,0.981991,0.741883


In [11]:
pd.Panel(predictions).to_pickle("dataset/keras_lstm_nsl_kdd_predictions.pkl")
scores.to_pickle("dataset/keras_lstm_nsl_kdd_scores.pkl")

In [12]:
pd.Panel(predictions)

<class 'pandas.core.panel.Panel'>
Dimensions: 5 (items) x 22544 (major_axis) x 2 (minor_axis)
Items axis: 30_1_16 to 30_1_8
Major_axis axis: 0 to 22543
Minor_axis axis: Attack_prob to Normal_prob